In [5]:
import pandas as pd
import requests as req
import time
from bs4 import BeautifulSoup as bs
from splinter import Browser
from selenium import webdriver
import re

In [6]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [7]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [8]:
news_url = "https://mars.nasa.gov/news/"
browser.visit(news_url)
html = browser.html

soup = bs(html, "html.parser")

article = soup.find("div", class_='list_text')
news_title = article.find("div", class_="content_title").text
news_p = article.find("div", class_ ="article_teaser_body").text
print(news_title)
print(news_p)

NASA's MAVEN Explores Mars to Understand Radio Interference at Earth
NASA’s MAVEN spacecraft has discovered “layers” and “rifts” in the electrically charged part of the upper atmosphere of Mars.


### JPL Mars Space Images - Featured Image
* Visit the url for JPL Featured Space Image.
* Use splinter to navigate the site and find the image url for the current Featured Mars Image.
* Assign the url string to a variable called featured_image_url.
* Make sure to find the image url to the full size .jpg image.
* Make sure to save a complete url string for this image.

In [9]:
# get something like this: https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16225_hires.jpg

image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(image_url)

browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(5)

browser.click_link_by_partial_text('more info')

html = browser.html
image_soup = bs(html, 'html.parser')

feat_img_url = image_soup.find('figure', class_='lede').a['href']
featured_image_url = f'https://www.jpl.nasa.gov{feat_img_url}'
print(featured_image_url)

/Users/stephanieroman/opt/anaconda3/lib/python3.7/site-packages/splinter/driver/webdriver/__init__.py:528: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA14934_hires.jpg


### Mars Weather
* Visit the Mars Weather twitter account and scrape the latest Mars weather tweet from the page. 
* Save the tweet text for the weather report as a variable called mars_weather.

In [10]:
# mars_weather = 'Sol 1801 (Aug 30, 2017), Sunny, high -21C/-5F, low -80C/-112F, pressure at 8.82 hPa, daylight 06:09-17:55'

#weather_url = 'https://twitter.com/marswxreport?lang=en'
#browser.visit(weather_url)

In [11]:
url_weather = 'https://twitter.com/marswxreport?lang=en'

response_weather = req.get(url_weather)

soup_weather = bs(response_weather.text, 'lxml')
soup_weather.find('div', class_='css-901oao r-hkyrab r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0')


weather_tweet = soup_weather.find('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text').text.strip()

print(weather_tweet)

InSight sol 422 (2020-02-03) low -90.2ºC (-130.3ºF) high -13.1ºC (8.5ºF)
winds from the SSE at 6.1 m/s (13.5 mph) gusting to 22.1 m/s (49.4 mph)
pressure at 6.20 hPapic.twitter.com/1wkFjO578P


### Mars Facts
* Visit the Mars Facts webpage.
* Use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
* Use Pandas to convert the data to a HTML table string.

In [12]:
facts_url = "https://space-facts.com/mars/"
browser.visit(facts_url)
html = browser.html

In [13]:
table = pd.read_html(facts_url)
mars_facts = table[1]

mars_facts.columns = ['Description','Mars', 'Earth']

mars_facts.set_index('Description', inplace=True)
mars_facts

,Mars,Earth
Description,,
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-153 to 20 °C,-88 to 58°C


In [14]:
mars_facts.to_html('table.html')

### Mars Hemispheres
* Visit the USGS Astrogeology site to obtain high resolution images for each of Mars's hemispheres. 
* You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.
* Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. 
* Use a Python dictionary to store the data using the keys img_url and title.
* Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

In [15]:

hemispheres_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemispheres_url)
html = browser.html

soup = bs(html, "html.parser")

hemisphere_image_urls = []

results = soup.find("div", class_ = "result-list" )
hemispheres = results.find_all("div", class_="item")

for hemisphere in hemispheres:
    title = hemisphere.find("h3").text
    title = title.replace("Enhanced", "")
    end_link = hemisphere.find("a")["href"]
    image_link = "https://astrogeology.usgs.gov/" + end_link    
    browser.visit(image_link)
    html = browser.html
    soup = bs(html, "html.parser")
    downloads = soup.find("div", class_="downloads")
    image_url = downloads.find("a")["href"]
    hemisphere_image_urls.append({"title": title, "img_url": image_url})

print(hemisphere_image_urls)

AttributeError: 'NoneType' object has no attribute 'find_all'